# Street Fighter No Delta
This notebook shows how to create the environment without the delta transformation. 

# Setup StreetFighter

In [ ]:
import retro

In [ ]:
retro.data.list_games()

In [ ]:
!python3 -m retro.import .

In [ ]:
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [ ]:
obs = env.reset()

In [ ]:
obs = env.reset()
done = False
for game in range(5):
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        print(reward)

# Setup Environment

In [ ]:
from gym import Env
from gym.spaces import Box, MultiBinary
import numpy as np
import cv2
import time
import numpy 


In [ ]:
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
        #self.score = 0
    
    def step(self, action):
        lst=[0]
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        
        # Preprocess frame from game
        frame_delta = obs 
#       - self.previous_frame
#         self.previous_frame = obs 

        
        #### Shape reward
        
        #1st Variation (just score)
        
        reward = info['score'] - self.score 
        self.score = info['score']
        
        
        #2nd Variation 
        
        #if  info['health']==0 and info['enemy_health']==0:
         #   reward=0
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
            
        #if  info['health']<0 and info['health']!=self.health and info['enemy_health']!=0:
         #   reward=(-176+((info['health'] - self.health)))*info['enemy_health']
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
            
        #elif info['enemy_health']<0 and info['enemy_health']!=self.enemy_health and info['health']!=0:
         #   reward=(176-(info['enemy_health'] - self.enemy_health))*(info['health'])
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health'] 
            
        #else: 
         #   reward=((info['health'] - self.health))-(info['enemy_health'] - self.enemy_health)
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
     
    
            
         #3rd Variation 
        
      #  rew2=((-176+(info['health']-self.health))* info['enemy_health'])
       # rew4=(info['health'] - self.health)*10 #dmg absorbed
        
        
        #if  info['health']==0 and info['enemy_health']==0:
         #   reward=0
          #  self.enemy_health = info['enemy_health']
            #self.health = info['health']
           # 
        #elif info['health']<0 and info['health']!=self.health and info['enemy_health']!=0:
         #   reward=rew2+(rew2*info['enemy_matches_won']/20)#καποια σταθερα
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
            #self.score = info['score']
            #
        #elif info['enemy_health']<0 and info['enemy_health']!=self.enemy_health and info['health']!=0:
         #   reward = info['score'] - self.score 
          #  self.score = info['score']
           # self.enemy_health = info['enemy_health']
            #self.health = info['health']
            #
        #else:
         #   if  (info['health']< self.health) and (info['enemy_health']==self.enemy_health): 
          #     self.enemy_health = info['enemy_health']
           #     self.health = info['health']
            #    self.score = info['score']
                
            #elif (info['health']< self.health) and (info['enemy_health']< self.enemy_health):
             #    reward= 10*(((info['health'] - self.health))-(info['enemy_health'] - self.enemy_health))#den pairnei score se isopalia
              #   self.enemy_health = info['enemy_health']
               #  self.health = info['health']   
                # print('double_damage',reward)
            #else:
             #    reward = (info['score'] - self.score)   
              #   self.score = info['score']
               #  self.enemy_health = info['enemy_health']
                # self.health = info['health']
      
        
        
        
        
        
        
        #######2/4nd Variation ((+) for doing damage or winning,(-) for taking damage or losing)
        
        #if info['health']<=0 and info['health']!=self.health and info['enemy_health']==0:
         #   reward=-176
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
            
        #elif info['enemy_health']<=0 and info['enemy_health']!=self.enemy_health and info['health']==0:
         #   reward=176
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health'] 
            
        #elif info['health']<=59 and info['health']!=self.health and info['enemy_health']!=0:
         #   reward=10*(info['health'] - self.health)-(info['enemy_health'] - self.enemy_health)
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']
            
        #elif info['enemy_health']<=59 and info['enemy_health']!=self.enemy_health and info['health']!=0:
         #   reward=(info['health'] - self.health)-10*(info['enemy_health'] - self.enemy_health)
          #  self.enemy_health = info['enemy_health']
           # self.health = info['health']  
            
       # else: 
        #    reward=((info['health'] - self.health))-(info['enemy_health'] - self.enemy_health)
         #   self.enemy_health = info['enemy_health']
          #  self.health = info['health']
              
                                                             
        return frame_delta, reward, done, info, 
    
    def render(self, *args, **kwargs): 
        self.game.render()
    
    def reset(self):
        self.previous_frame = np.zeros(self.game.observation_space.shape)
        
        # Frame delta
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        
        # Create initial variables
        self.score = 0
        
        #self.enemy_health=0
        #self.health=0

        return obs
    
    def preprocess(self, observation): 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (84,84,1))
        return state
    
    def close(self): 
        self.game.close()

In [ ]:
env.close()


In [ ]:
env = StreetFighter()

In [ ]:
env.observation_space.shape

In [ ]:
obs = env.reset()
done = False
for game in range(5):
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        if reward!=0:
           print(reward,info['health'],info['enemy_health'])
        time.sleep(0.01)

In [ ]:
!pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
!python3 -m pip install stable-baselines3[extra] 

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# Import A2C for algos
from stable_baselines3 import A2C
# Evaluate Policy
from stable_baselines3.common.evaluation import evaluate_policy
# Import wrappers
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import os 
import optuna

In [ ]:
LOG_DIR = './logs/'
OPT_DIR = './opt_nodelta'

# Setup Callback

In [ ]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train_a2c_rew1_rmsprop(opt)/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# Train Model

In [ ]:
env.close()

In [ ]:
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
model_params = {'n_steps': 50, 'gamma': 0.99, 'gae_lambda':1, 'learning_rate': 7e-5, 'vf_coef': 0.5,'ent_coef': 0.0,'max_grad_norm':0.5, 'rms_prop_eps':1e-05 }
# opt1 A2C-21 (REW1)  model_params = {'n_steps': 50, 'gamma': 0.99, 'gae_lambda':1, 'learning_rate': 7e-5, 'vf_coef': 0.5,'ent_coef': 0.0,'max_grad_norm':0.5, 'rms_prop_eps':1e-05 }
#default model_params = {'n_steps': 5, 'gamma': 0.99, 'gae_lambda':1, 'learning_rate': 7e-4, 'vf_coef': 0.5,'ent_coef': 0.0,'max_grad_norm':0.5, 'rms_prop_eps':1e-05 }

In [ ]:
model_params

In [ ]:
import torch
from torch.optim import Optimizer

In [ ]:
class RMSpropTF(Optimizer):
    """Implements RMSprop algorithm (TensorFlow style epsilon)
    NOTE: This is a direct cut-and-paste of PyTorch RMSprop with eps applied before sqrt
    and a few other modifications to closer match Tensorflow for matching hyper-params.
    Noteworthy changes include:
    1. Epsilon applied inside square-root
    2. square_avg initialized to ones
    3. LR scaling of update accumulated in momentum buffer
    Proposed by G. Hinton in his
    `course <http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf>`_.
    The centered version first appears in `Generating Sequences
    With Recurrent Neural Networks <https://arxiv.org/pdf/1308.0850v5.pdf>`_.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-2)
        momentum (float, optional): momentum factor (default: 0)
        alpha (float, optional): smoothing (decay) constant (default: 0.9)
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-10)
        centered (bool, optional) : if ``True``, compute the centered RMSProp,
            the gradient is normalized by an estimation of its variance
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        decoupled_decay (bool, optional): decoupled weight decay as per https://arxiv.org/abs/1711.05101
        lr_in_momentum (bool, optional): learning rate scaling is included in the momentum buffer
            update as per defaults in Tensorflow
    """

    def __init__(self, params, lr=1e-2, alpha=0.9, eps=1e-10, weight_decay=0, momentum=0., centered=False,
                 decoupled_decay=False, lr_in_momentum=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= momentum:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if not 0.0 <= alpha:
            raise ValueError("Invalid alpha value: {}".format(alpha))

        defaults = dict(
            lr=lr, momentum=momentum, alpha=alpha, eps=eps, centered=centered, weight_decay=weight_decay,
            decoupled_decay=decoupled_decay, lr_in_momentum=lr_in_momentum)
        super(RMSpropTF, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RMSpropTF, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('momentum', 0)
            group.setdefault('centered', False)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad
                if grad.is_sparse:
                    raise RuntimeError('RMSprop does not support sparse gradients')
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    state['square_avg'] = torch.ones_like(p)  # PyTorch inits to zero
                    if group['momentum'] > 0:
                        state['momentum_buffer'] = torch.zeros_like(p)
                    if group['centered']:
                        state['grad_avg'] = torch.zeros_like(p)

                square_avg = state['square_avg']
                one_minus_alpha = 1. - group['alpha']

                state['step'] += 1

                if group['weight_decay'] != 0:
                    if group['decoupled_decay']:
                        p.mul_(1. - group['lr'] * group['weight_decay'])
                    else:
                        grad = grad.add(p, alpha=group['weight_decay'])

                # Tensorflow order of ops for updating squared avg
                square_avg.add_(grad.pow(2) - square_avg, alpha=one_minus_alpha)
                # square_avg.mul_(alpha).addcmul_(grad, grad, value=1 - alpha)  # PyTorch original

                if group['centered']:
                    grad_avg = state['grad_avg']
                    grad_avg.add_(grad - grad_avg, alpha=one_minus_alpha)
                    avg = square_avg.addcmul(grad_avg, grad_avg, value=-1).add(group['eps']).sqrt_()  # eps in sqrt
                    # grad_avg.mul_(alpha).add_(grad, alpha=1 - alpha)  # PyTorch original
                else:
                    avg = square_avg.add(group['eps']).sqrt_()  # eps moved in sqrt

                if group['momentum'] > 0:
                    buf = state['momentum_buffer']
                    # Tensorflow accumulates the LR scaling in the momentum buffer
                    if group['lr_in_momentum']:
                        buf.mul_(group['momentum']).addcdiv_(grad, avg, value=group['lr'])
                        p.add_(-buf)
                    else:
                        # PyTorch scales the param update by LR
                        buf.mul_(group['momentum']).addcdiv_(grad, avg)
                        p.add_(buf, alpha=-group['lr'])
                else:
                    p.addcdiv_(grad, avg, value=-group['lr'])

        return loss

In [ ]:
#model = A2C('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)
model = A2C('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params, policy_kwargs=dict(optimizer_class=RMSpropTF))

In [ ]:
model.learn(total_timesteps=10000000, callback=callback)

In [ ]:
env.close()

# Evaluate the Model

In [ ]:
model = A2C.load('/media/user/My Passport/babis arxeia/train_a2c_rew1/best_model_8000000.zip')

In [ ]:
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10, render=True)#n_eval_episodes=10

In [ ]:
mean_reward

In [ ]:
env.close()

# Test out the Model

In [ ]:
import time

In [ ]:
for episode in range(1): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        env.render()
       # time.sleep(0.01)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(total_reward, episode))
    time.sleep(2)